### Mini Multi-Head Dot-Product Attention’dan başlıyoruz.

Bu küçük versiyon:

* Multi-head ama minimal parametre ile

* Residual, LayerNorm ve dropout yok

* Mask ve causal opsiyonel değil, sadece temel mantığı gösteriyor

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MiniMultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=2):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q/K/V projeksiyonları
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, x):
        B, L, E = x.shape
        
        # 1️⃣ Q/K/V projeksiyonları ve head bölünmesi
        Q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        
        # 2️⃣ Skor hesaplama
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        attn = F.softmax(scores, dim=-1)
        
        # 3️⃣ Ağırlıklı toplam
        context = torch.matmul(attn, V)
        
        # 4️⃣ Head birleştirme
        context = context.transpose(1,2).contiguous().view(B, L, E)
        out = self.out_proj(context)
        return out, attn

# Test
B, L, E = 2, 4, 16
x = torch.randn(B, L, E)

mini_mha = MiniMultiHeadAttention(E, num_heads=2)
y, attn_map = mini_mha(x)
print("Output:", y.shape)        # (B, L, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L, L)


### Bu mini versiyon:

* Minimal multi-head attention mantığını gösteriyor

* Her head kendi attention’ını öğreniyor

* Henüz residual, layernorm, dropout veya mask yok

---
## İlk olarak mini Multi-Head Attention’ı residual ve layernorm ile güçlendireceğiz.
---

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MiniMHA_Residual(nn.Module):
    def __init__(self, embed_dim, num_heads=2):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q/K/V projeksiyonları
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # LayerNorm
        self.layernorm = nn.LayerNorm(embed_dim)
        
    def forward(self, x):
        B, L, E = x.shape
        
        # Q/K/V projeksiyonları
        Q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        
        # Skor ve ağırlıklı toplam
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        attn = F.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)
        
        # Head birleştirme
        context = context.transpose(1,2).contiguous().view(B, L, E)
        out = self.out_proj(context)
        
        # Residual + LayerNorm
        out = self.layernorm(x + out)
        return out, attn

# Test
B, L, E = 2, 4, 16
x = torch.randn(B, L, E)

mha_res = MiniMHA_Residual(E, num_heads=2)
y, attn_map = mha_res(x)
print("Output:", y.shape)        # (B, L, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L, L)


Output: torch.Size([2, 4, 16])
Attention map: torch.Size([2, 2, 4, 4])


## 💡 Açıklama:

* Residual: Girdiyi çıktıya ekleyerek öğrenmeyi kolaylaştırır

* LayerNorm: Stabil ve hızlı öğrenme sağlar

---
##  Adım — Dropout ve Padding Mask

### 🔹 Amaç
- **Dropout:** Overfitting’i azaltmak için attention ağırlıklarını ve çıkışı rastgele sıfırlamak.  
- **Padding Mask:** Seq2Seq veya değişken uzunluklu dizilerde geçersiz token’lara dikkat edilmemesi.

### 🔹 Mantık
1. **Scores Hesaplama:**  
\[
\text{scores} = \frac{Q K^T}{\sqrt{d_k}}
\]

2. **Mask Uygulama:**  
Padding token’ları `-inf` ile softmax’ten önce devre dışı bırakılır.

3. **Softmax → attn:**  
Skorlar normalize edilir.

4. **Dropout:**  
- attn üzerinde  
- final output üzerinde  
overfitting’i azaltmak için uygulanır.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MiniMHA_Residual_Dropout(nn.Module):
    def __init__(self, embed_dim, num_heads=2, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q/K/V projeksiyonları
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # Dropout + LayerNorm
        self.dropout = nn.Dropout(dp)
        self.layernorm = nn.LayerNorm(embed_dim)
        
    def forward(self, x, mask=None):
        B, L, E = x.shape
        
        # 1️⃣ Q/K/V projeksiyonları ve head bölünmesi
        Q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        
        # 2️⃣ Skor hesaplama
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        
        # 3️⃣ Padding mask
        if mask is not None:
            mask_exp = mask.unsqueeze(1).unsqueeze(1)  # (B,1,1,L)
            scores = scores.masked_fill(mask_exp == 0, float('-inf'))
        
        # 4️⃣ Softmax ve dropout
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        # 5️⃣ Ağırlıklı toplam ve head birleştirme
        context = torch.matmul(attn, V)
        context = context.transpose(1,2).contiguous().view(B, L, E)
        out = self.out_proj(context)
        out = self.dropout(out)
        
        # 6️⃣ Residual + LayerNorm
        out = self.layernorm(x + out)
        return out, attn

# Test
B, L, E = 2, 4, 16
x = torch.randn(B, L, E)
mask = torch.tensor([[1,1,1,0],[1,1,0,0]])  # padding mask

mha_dp = MiniMHA_Residual_Dropout(E, num_heads=2, dp=0.1)
y, attn_map = mha_dp(x, mask=mask)
print("Output:", y.shape)        # (B, L, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L, L)


Output: torch.Size([2, 4, 16])
Attention map: torch.Size([2, 2, 4, 4])


## Multi-Head Dot-Product Attention + Residual + LayerNorm

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DotProductMHA_Residual(nn.Module):
    def __init__(self, embed_dim, num_heads=2):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q/K/V projeksiyonları
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # LayerNorm
        self.layernorm = nn.LayerNorm(embed_dim)
        
    def forward(self, x):
        B, L, E = x.shape
        
        # 1️⃣ Q/K/V projeksiyonları ve head bölünmesi
        Q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        
        # 2️⃣ Skor hesaplama (Dot-Product)
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        attn = F.softmax(scores, dim=-1)
        
        # 3️⃣ Ağırlıklı toplam ve head birleştirme
        context = torch.matmul(attn, V)
        context = context.transpose(1,2).contiguous().view(B, L, E)
        out = self.out_proj(context)
        
        # 4️⃣ Residual + LayerNorm
        out = self.layernorm(x + out)
        return out, attn

# Test
B, L, E = 2, 4, 16
x = torch.randn(B, L, E)

dot_mha = DotProductMHA_Residual(E, num_heads=2)
y, attn_map = dot_mha(x)
print("Output:", y.shape)        # (B, L, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L, L)


Output: torch.Size([2, 4, 16])
Attention map: torch.Size([2, 2, 4, 4])


##  Adım — Causal / Look-Ahead Mask ve Autoregressive Destek

### 🔹 Amaç
- Autoregressive görevlerde her token yalnızca kendisinden önceki token’lara bakabilir.  
- Gelecek token’lar maskelenir (`-inf` ile softmax öncesi).  
- Böylece dil modelleme ve seq2seq görevleri için uygun hale gelir.

### 🔹 Mantık
1. Q/K/V projeksiyonları → head’lere bölünür  
2. Skor:  
\[
\text{scores} = \frac{Q K^T}{\sqrt{d_k}}
\]
3. **Padding Mask** uygulanır (opsiyonel)  
4. **Causal Mask** uygulanır  
5. Softmax → attn  
6. Dropout → attn ve output  
7. Head’ler birleştirilir  
8. Residual + LayerNorm → son çıktı


In [5]:
class DotProductMHA_Causal(nn.Module):
    def __init__(self, embed_dim, num_heads=2, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        self.dropout = nn.Dropout(dp)
        self.layernorm = nn.LayerNorm(embed_dim)
        
    def forward(self, x, padding_mask=None):
        B, L, E = x.shape
        Q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        
        # Skor
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        
        # Padding mask
        if padding_mask is not None:
            mask_exp = padding_mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask_exp == 0, float('-inf'))
        
        # Causal mask
        causal_mask = torch.tril(torch.ones(L, L, device=x.device)).unsqueeze(0).unsqueeze(0)
        scores = scores.masked_fill(causal_mask == 0, float('-inf'))
        
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        context = torch.matmul(attn, V)
        context = context.transpose(1,2).contiguous().view(B, L, E)
        out = self.out_proj(context)
        out = self.dropout(out)
        
        out = self.layernorm(x + out)
        return out, attn

# Test
B, L, E = 2, 4, 16
x = torch.randn(B, L, E)
padding_mask = torch.tensor([[1,1,1,0],[1,1,0,0]])
mha_causal = DotProductMHA_Causal(E, num_heads=2, dp=0.1)
y, attn_map = mha_causal(x, padding_mask=padding_mask)
print("Output:", y.shape)
print("Attention map:", attn_map.shape)


Output: torch.Size([2, 4, 16])
Attention map: torch.Size([2, 2, 4, 4])


##  Adım — Opsiyonel Gating / Fusion

### 🔹 Amaç
- Attention çıktısı ile orijinal input’u akıllıca birleştirmek.  
- Model, hangi bilgiyi daha çok kullanacağını öğrenir.  
- Özellikle çok katmanlı veya encoder-decoder yapılarında faydalıdır.

### 🔹 Mantık
1. Attention çıktısı `context` ve orijinal input `x` alınır  
2. Gating ağı (sigmoid aktivasyonlu) ile hangi kısımların kullanılacağı belirlenir:

\[
g = \sigma(W_g [context; x])
\]

3. Fusion uygulanır:

\[
y = g * context + (1 - g) * x
\]

4. Residual + LayerNorm uygulanır (opsiyonel)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MiniMHA_Gating(nn.Module):
    def __init__(self, embed_dim, num_heads=2, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q/K/V projeksiyonları
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # LayerNorm ve Dropout
        self.layernorm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dp)
        
        # Gating ağı
        self.gate = nn.Linear(embed_dim*2, embed_dim)
        
    def forward(self, x, mask=None):
        B, L, E = x.shape
        
        # Q/K/V projeksiyonları ve head bölünmesi
        Q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        
        # Skor ve attention
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        if mask is not None:
            mask_exp = mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask_exp == 0, float('-inf'))
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        # Weighted sum ve head birleştirme
        context = torch.matmul(attn, V)
        context = context.transpose(1,2).contiguous().view(B, L, E)
        context = self.out_proj(context)
        context = self.dropout(context)
        
        # Gating / Fusion
        combined = torch.cat([context, x], dim=-1)
        g = torch.sigmoid(self.gate(combined))
        out = g * context + (1 - g) * x
        
        # Residual + LayerNorm
        out = self.layernorm(out)
        return out, attn

# Test
B, L, E = 2, 4, 16
x = torch.randn(B, L, E)
mask = torch.tensor([[1,1,1,0],[1,1,0,0]])

mha_gating = MiniMHA_Gating(E, num_heads=2, dp=0.1)
y, attn_map = mha_gating(x, mask=mask)

print("Output:", y.shape)         # (B, L, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L, L)


Output: torch.Size([2, 4, 16])
Attention map: torch.Size([2, 2, 4, 4])


## 💡 Açıklama:

* context ve x concat edilip gating ağı ile fusion uygulanıyor

* Sigmoid ile hangi bilgi ne kadar kullanılacak öğreniliyor

* Dropout + LayerNorm ile stabil ve overfitting’e karşı dayanıklı hâle getiriliyor

* Artık Mini Multi-Head Dot-Product Attention tam esnek ve güçlü hâlde 😎

## Mini Multi-Head Dot-Product Attention + Causal Mask + Padding Mask + Gating/Fusion kombinasyonunu birleştirip tam üretime hazır hâle getirelim.

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MiniMHA_Full(nn.Module):
    def __init__(self, embed_dim, num_heads=2, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q/K/V projeksiyonları
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # LayerNorm ve Dropout
        self.layernorm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dp)
        
        # Gating ağı
        self.gate = nn.Linear(embed_dim*2, embed_dim)
        
    def forward(self, x, padding_mask=None):
        B, L, E = x.shape
        
        # Q/K/V projeksiyonları ve head bölünmesi
        Q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1,2)
        
        # Skor ve attention
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        
        # Padding mask
        if padding_mask is not None:
            mask_exp = padding_mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask_exp == 0, float('-inf'))
        
        # Causal mask
        causal_mask = torch.tril(torch.ones(L, L, device=x.device)).unsqueeze(0).unsqueeze(0)
        scores = scores.masked_fill(causal_mask == 0, float('-inf'))
        
        # Softmax ve dropout
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        # Weighted sum ve head birleştirme
        context = torch.matmul(attn, V)
        context = context.transpose(1,2).contiguous().view(B, L, E)
        context = self.out_proj(context)
        context = self.dropout(context)
        
        # Gating / Fusion
        combined = torch.cat([context, x], dim=-1)
        g = torch.sigmoid(self.gate(combined))
        out = g * context + (1 - g) * x
        
        # Residual + LayerNorm
        out = self.layernorm(out)
        return out, attn

# Test
B, L, E = 2, 4, 16
x = torch.randn(B, L, E)
padding_mask = torch.tensor([[1,1,1,0],[1,1,0,0]])

mha_full = MiniMHA_Full(E, num_heads=2, dp=0.1)
y, attn_map = mha_full(x, padding_mask=padding_mask)

print("Output:", y.shape)         # (B, L, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L, L)


Output: torch.Size([2, 4, 16])
Attention map: torch.Size([2, 2, 4, 4])


## 💡 Açıklama:

* Q/K/V projeksiyonları → Multi-head Dot-Product Attention

* Padding Mask → geçersiz token’ları devre dışı bırakır

* Causal Mask → autoregressive görevler için geleceğe bakışı engeller

* Dropout → attn ve output üzerinde overfitting’i önler

* Gating / Fusion → context ve x akıllıca birleştirilir

* Residual + LayerNorm → stabil ve hızlı öğrenme
